# Simple Reinforcement Learning in Tensorflow Part 1:

## The Multi-armed bandit

이 튜토리얼은 여러 손잡이 슬롯머신 문제를 풀 수 있는 정책-그라디언트 기반의 에이전트를 만드는 방법에 대한 단순한 예제를 담고있다.

In [1]:
import tensorflow as tf
import numpy as np

## The Bandits

여기서 슬롯 머신을 정의한다. 이 예제에서 우리는 4개의 손잡이를 가진 슬롯머신을 사용한다. 이 pullBandit 함수는 0의 평균을 갖는 정규분포로부터 무작위 값을 생성한다. 이 슬롯머신의 수가 더 작을 수록, 양의 보상을 더 자주 반환할 것이다. 우리는 우리 에이전트가 양의 보상을 줄 슬롯머신을 항상 선택하는 것을 학습하기를 원한다.

In [2]:
# 리스트는 우리의 슬롯머신, 현재 슬롯머신 4(index는 3) 은 가장 자주 양의 보상을 제공하는 설정이다.
bandits = [0.2,0,-0.2,-5]
# 슬롯머신의 수
num_bandits = len(bandits)


def pullBandit(bandit):
    # 0의 평균을 갖는 정규분포에서 랜덤한 숫자를 뽑는다. 
    result = np.random.randn(1)
    
    # 만약 이 랜덤값이 위의 슬롯머신의 고정된 확률보다 크다면 양의 보상을 반환한다.
    if result > bandit:
        return 1
    
    # 그렇지 않다면, 음의 보상을 반환한다.
    else:
        return -1

## The Agent

아래의 코드는 우리의 단순한 신경망 에이전트를 만든다. 이 슬롯머신 각각에 대한 값들의 집합으로 구성되어 있다. 각 값은 슬롯머신을 선택하는 것으로부터 반환값의 추정치이다. 우리는 선택된 행동의 값이 받은 보상을 향해 이동하도록 에이전트를 업데이트하기위해 정책 그라디언트 방법을 사용한다.

In [3]:
tf.reset_default_graph()

# 아래 두줄은 신경망의 feed-forward 부분을 만든다. 이것이 실제 선택을 한다.
weights = tf.Variable(tf.ones([num_bandits]))
# 주어진 가중치 중에서 최대값의 인덱스를 뽑아 선택된 행동이라 생각한다.
chosen_action = tf.argmax(weights,0)


# 다음 여섯줄은 학습 과정을 만든다. 우리는 보상을 받고 신경망속에 선택된 행동을 보내 손실을 계산하고, 
# 신경망을 업데이트하기 위해 이것을 이용한다.

# 보상을 받는 부분
reward_holder = tf.placeholder(shape=[1],dtype=tf.float32)
# 행동을 받는 부분
action_holder = tf.placeholder(shape=[1],dtype=tf.int32)
# 가중치 중에서 행동에 해당하는 가중치를 가져오는 부분
responsible_weight = tf.slice(weights,action_holder,[1])
# 손실함수를 계산하는 부분 가중치와 보상을 곱해서 구한다.
loss = -(tf.log(responsible_weight)*reward_holder)
# 최적화에 사용할 방법을 고르는 부분
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
# 업데이트 부분
update = optimizer.minimize(loss)

## Training the Agent

우리는 우리의 환경에서 행동들을 선택하는 것과 보상을 받는 것으로써 우리의 에이전트를 학습할 것이다. 보상과 행동을 사용해, 우리는 시간에 걸쳐 가장 높은 보상을 산출해내는 행동들을 더 자주 선택하게 하기 위해 우리의 신경망을 적절히 업데이트하는 방법에 대해 알 수 있다.

In [5]:
total_episodes = 1000 # 에이전트를 학습하기 위한 총 에피소드 수를 선택한다 
total_reward = np.zeros(num_bandits) # 슬롯머신의 스코어보드를 0으로 설정한다 
e = 0.1 # 무작위 행동을 선택할 확률을 설정한다.

init = tf.global_variables_initializer()

# 텐서플로 그래프를 실행한다
with tf.Session() as sess:
    # 모든 변수를 초기화한다.
    sess.run(init)
    # 에피소드 수를 기록
    i = 0
    # 에피소드 수가 총 에피소드(1000)을 넘지 않으면 계속 수행
    while i < total_episodes:
        
        # 우리의 신경망으로부터 하나나 랜덤한 행동을 선택한다
        # e보다 작은 경우엔 4개중 랜덤한 행동을 한다.
        # 아니면 가중치중에 가장 큰 가중치를 갖는 것의 인덱스로 행동을 고른다.
        if np.random.rand(1) < e:
            action = np.random.randint(num_bandits)
        else:
            action = sess.run(chosen_action)
            
        reward = pullBandit(bandits[action]) # 슬롯머신의 하나를 선택하는 것으로부터 우리의 보상을 얻는다
        
        # 신경망을 업데이트 하는 부분
        # 위에서 주어진 행동에 대한 미래의 보상과 행동으로 가중치를 업데이트한다.
        _,resp,ww = sess.run([update,responsible_weight,weights], feed_dict={reward_holder:[reward],action_holder:[action]})
        
        # 에피소드 동안의 보상을 기록한다.
        total_reward[action] += reward
        
        # 50번째 마다 출력한다.
        if i % 50 == 0:
            print( "Running reward for the " + str(num_bandits) + " bandits: " + str(total_reward))
        
        # 에피소드 수를 늘려준다.
        i+=1

# 가장 좋은 슬롯머신을 고른다.
print ("The agent thinks bandit " + str(np.argmax(ww)+1) + " is the most promising....")

# 가중치중에 제일 큰 값이 슬롯머신 확률중에 제일 낮으면 맞았다고 하고, 아니면 틀렸다고 한다.
if np.argmax(ww) == np.argmax(-np.array(bandits)):
    print ("...and it was right!")
else:
    print ("...and it was wrong!")

Running reward for the 4 bandits: [-1.  0.  0.  0.]
Running reward for the 4 bandits: [  3.  -1.  -1.  34.]
Running reward for the 4 bandits: [  3.  -1.  -1.  84.]
Running reward for the 4 bandits: [   4.   -3.   -1.  131.]
Running reward for the 4 bandits: [   4.   -4.   -1.  178.]
Running reward for the 4 bandits: [   3.   -4.   -2.  224.]
Running reward for the 4 bandits: [   2.   -5.   -3.  269.]
Running reward for the 4 bandits: [   4.   -4.   -2.  313.]
Running reward for the 4 bandits: [   5.   -5.    1.  358.]
Running reward for the 4 bandits: [   5.   -6.    2.  402.]
Running reward for the 4 bandits: [   3.   -6.    4.  444.]
Running reward for the 4 bandits: [   3.   -7.    4.  491.]
Running reward for the 4 bandits: [   4.   -6.    4.  535.]
Running reward for the 4 bandits: [   3.   -6.    6.  582.]
Running reward for the 4 bandits: [   3.   -4.    5.  627.]
Running reward for the 4 bandits: [   2.   -4.    4.  673.]
Running reward for the 4 bandits: [   3.   -3.    6.  71